# Chat Completions

Chat models take a series of messages as input, and return a model-generated message as output.
The main input is the messages parameter. **Messages must be an array of message objects, where each object has a role (either "system", "user") and content** (the content of the message). 

In [ ]:
# if needed, install and/or upgrade to the latest version of the OpenAI Python library
#%pip install --upgrade openai

In [3]:
# import openai
# from openai import AzureOpenAI
# import os 
# from azure.identity import ManagedIdentityCredential

# default_credential=ManagedIdentityCredential(client_id="d30cba06-04c1-4065-a91d-8b7ce3b07b78")
# token=default_credential.get_token("https://cognitiveservices.azure.com/.default")
# Resource_endpoint="https://openaiykus.openai.azure.com/"

# client = AzureOpenAI(
#   azure_endpoint = Resource_endpoint, 
#   api_key=token.token,  
#   api_version="2023-05-15"
# )

In [1]:
import os
import openai
from openai import AzureOpenAI
from dotenv import load_dotenv

# Set up Azure OpenAI
load_dotenv("credentials.env")

openai.api_type = "azure"
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2024-02-01",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

In [2]:
messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Can you write a welcome page for The Enstar Group company website ?"}]

In [3]:
response = client.chat.completions.create(
    model="gpt-4-0125-Preview", # model = "deployment_name".
    messages=messages
)
print(response.choices[0].message.content)

Certainly! Below is an example welcome page for The Enstar Group company website. Note that this is a general template and should be customized to align with Enstar's current branding, tone, values, and factual company details.

---

### Welcome to Enstar Group

#### Navigating the Present, Shaping the Future

Welcome to Enstar Group, a global leader in insurance and reinsurance solutions. Since our inception, we've been at the forefront of innovation, focusing on a core mission to manage and resolve insurance and reinsurance portfolios efficiently, with integrity, and with a forward-thinking approach. Our comprehensive services, unparalleled expertise, and commitment to excellence have positioned us as trusted partners in the industry.

#### Who We Are

Enstar Group specializes in managing insurance and reinsurance run-off companies and portfolios and provides capital release solutions. With our extensive knowledge and experience, we've become experts in acquisitions, reinsurance, and

In [4]:
response

ChatCompletion(id='chatcmpl-9Rp5zt7zvjnWGsUDVl2IuE3hJDGxQ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Certainly! Below is an example welcome page for The Enstar Group company website. Note that this is a general template and should be customized to align with Enstar's current branding, tone, values, and factual company details.\n\n---\n\n### Welcome to Enstar Group\n\n#### Navigating the Present, Shaping the Future\n\nWelcome to Enstar Group, a global leader in insurance and reinsurance solutions. Since our inception, we've been at the forefront of innovation, focusing on a core mission to manage and resolve insurance and reinsurance portfolios efficiently, with integrity, and with a forward-thinking approach. Our comprehensive services, unparalleled expertise, and commitment to excellence have positioned us as trusted partners in the industry.\n\n#### Who We Are\n\nEnstar Group specializes in managing insurance and reinsurance

In [5]:
messages.append({"role": "system", "content": response.choices[0].message.content})

In [6]:
messages.append({"role": "user", "content": "Can you summarize our conversation?"})

In [7]:
print(messages)

[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Can you write a welcome page for The Enstar Group company website ?'}, {'role': 'system', 'content': "Certainly! Below is an example welcome page for The Enstar Group company website. Note that this is a general template and should be customized to align with Enstar's current branding, tone, values, and factual company details.\n\n---\n\n### Welcome to Enstar Group\n\n#### Navigating the Present, Shaping the Future\n\nWelcome to Enstar Group, a global leader in insurance and reinsurance solutions. Since our inception, we've been at the forefront of innovation, focusing on a core mission to manage and resolve insurance and reinsurance portfolios efficiently, with integrity, and with a forward-thinking approach. Our comprehensive services, unparalleled expertise, and commitment to excellence have positioned us as trusted partners in the industry.\n\n#### Who We Are\n\nEnstar Group specializes in 

In [9]:
response = client.chat.completions.create(
    model="gpt-4-0125-Preview", # model = "deployment_name".
    messages=messages
)
print(response.choices[0].message.content)

Certainly! In our conversation, you requested a welcome page for The Enstar Group company website. I provided a customizable welcome page template that includes sections on:

1. A greeting and brief introduction to Enstar Group, highlighting its role as a leader in insurance and reinsurance solutions.
2. A "Who We Are" section detailing Enstar Group's specialization in managing insurance and reinsurance run-off companies and portfolios, along with a mention of global operations.
3. The company's commitments to clients, innovation, its people, and ethical responsibility.
4. An overview of services offered by Enstar Group, such as legacy solutions, capital solutions, acquisitions, and management services.
5. An invitation for visitors to explore the website further, follow Enstar Group on social media, and subscribe to its newsletter, along with a closing statement emphasizing the company's dedication to excellence and innovation.

I emphasized that the provided content should be customi

In [10]:
# Instead of appending, writing messages in the SDK
response = client.chat.completions.create(
    model="gpt-4-0125-Preview", # model = "deployment_name".
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Can you write a welcome page for The Enstar Group company website ?"},
        {"role": "system", "content": "Welcome to The Enstar Group, a company that integrates creativity, innovation, and cutting-edge technology to create unique and exceptional customer experiences."},
        {"role": "user", "content": "Can you summarize our conversation?"},
    ]
)

print(response.choices[0].message.content)

Certainly! You requested a welcome page message for The Enstar Group company website, and I provided a brief introduction highlighting the company's focus on creativity, innovation, and the use of advanced technology to enhance customer experiences.


# PII Redaction

In [11]:
content= f"""You are a helpful assistant that answers the given question. You must present both response and original question in your output after redacting any PII data such as name, address, date of birth, medical records, and financial records.
 
Format the output in JSON by using "Original Question" and "Response" as a key. The original question should be the user input verbatim."""

In [12]:

response = client.chat.completions.create(
    model="gpt-4-0125-Preview", # model = "deployment_name".
    messages=[
        {"role": "system", "content": content},
        {"role": "user", "content": "Hi, I am Yeliz, based in Cambridge. Can you write a poem for me?"}
    ]
)

print(response.choices[0].message.content)

{
  "Original Question": "Hi, I am [Name Redacted], based in [Location Redacted]. Can you write a poem for me?",
  "Response": "Certainly! Here is a poem for you:\n\nIn a town where dreams intertwine,\nBeneath the academic spires that shine,\nThere lies a heart so bold and fine,\nSeeking wisdom's treasure, a quest divine.\n\nThrough cobbled streets and history’s embrace,\nPast rivers of knowledge that time cannot erase,\nA spirit wanders, a gentle trace,\nOf longing for learning, and life’s sweet taste.\n\nIn this haven of thought, where bright minds meet,\nIdeas flourish, and hearts beat,\nA dedication to pursuit, no retreat,\nIn Cambridge’s embrace, a journey complete.\n\nMay your days be filled with curious delight,\nUnder stars that guide you through the night,\nMay your path be clear, your future bright,\nIn the quest for wisdom, and truth’s light."
}


### **GPT-4 TURBO  Reproducable Outputs with Seed Parameter**

In [14]:
for i in range(3):
  print(f'Story Version {i + 1}\n---')
    
  response = client.chat.completions.create(
    model="gpt-4-0125-Preview", # Model = should match the deployment name you chose for your 1106-preview model deployment
    seed=42,
    temperature=0.5,
    max_tokens =50, 
    messages=[
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": "Tell me a story about how the universe began?"}
    ]
  )
  
  print(response.choices[0].message.content)
  print("---\n")


Story Version 1
---
In the vast expanse of nothingness, where time and space had yet to find their form, a singular event sparked the creation of everything we know—the universe's beginning. This is a tale not of science or fact, but of imagination, inspired
---

Story Version 2
---
In the vast expanse of nothingness, where time and space had yet to find their form, a singular event sparked the creation of everything we know—the universe's beginning. This is a tale not of science or fact, but of imagination, inspired
---

Story Version 3
---
In the vast expanse of nothingness, where time and space had yet to find their form, a singular event sparked the creation of everything we know—the universe's beginning. This is a tale not of science or fact, but of imagination, inspired
---

